In [ ]:
import pandas as pd
import geopandas as gpd
import leafmap.foliumap as leafmap

In [2]:
repo = "https://raw.githubusercontent.com/opengeos/maxar-open-data/master/"

In [14]:
datasets = pd.read_csv(f"{repo}datasets.csv")
dataset = "Nepal-Floods-Sept-2024"
df = pd.read_csv(f"{repo}datasets/{dataset}.tsv", sep="\t")
df.head()

,datetime,platform,gsd,ard_metadata_version,catalog_id,utm_zone,quadkey,view:off_nadir,view:azimuth,view:incidence_angle,...,proj:epsg,grid:code,proj:bbox,tile:data_area,tile:clouds_area,tile:clouds_percent,visual,ms_analytic,pan_analytic,data-mask
0,2022-03-26 04:54:40+00:00,WV02,0.58,0.0.1,10300100D0939200,45,31331122122,28.1,51.5,58.2,...,32645,MXRA-Z45-031331122122,"361069.64111328125,3059843.75,365156.25,306337...",14.2,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...
1,2022-03-26 04:54:40+00:00,WV02,0.58,0.0.1,10300100D0939200,45,31331122123,27.9,51.2,58.4,...,32645,MXRA-Z45-031331122123,"364843.75,3059843.75,370156.25,3063343.4747489337",18.4,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...
2,2022-03-26 04:54:40+00:00,WV02,0.58,0.0.1,10300100D0939200,45,31331122132,27.6,50.8,58.7,...,32645,MXRA-Z45-031331122132,"369843.75,3059843.75,375156.25,3063295.400815661",18.2,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...
3,2022-03-26 04:54:41+00:00,WV02,0.58,0.0.1,10300100D0939200,45,31331122133,27.4,50.4,58.9,...,32645,MXRA-Z45-031331122133,"374843.75,3059843.75,380156.25,3063247.3268823884",17.9,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...
4,2022-03-26 04:54:41+00:00,WV02,0.58,0.0.1,10300100D0939200,45,31331122300,28.1,51.2,58.1,...,32645,MXRA-Z45-031331122300,"361006.16455078125,3054843.75,365156.25,306015...",21.7,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...,https://maxar-opendata.s3.amazonaws.com/events...


In [ ]:
from pyproj import Proj, transform

# Define the projections
wgs84 = Proj(init='epsg:4326')  # WGS 84 (lat/lon)
utm_32645 = Proj(init='epsg:32645')  # UTM zone 45N

# Coordinates in EPSG:4326 (Longitude, Latitude)
minx, miny = 85.19955572969292, 27.555855160395794
maxx, maxy = 85.57366264367934, 27.847521076376147

# Transform the coordinates to EPSG:32645
minx_utm, miny_utm = transform(wgs84, utm_32645, minx, miny)
maxx_utm, maxy_utm = transform(wgs84, utm_32645, maxx, maxy)

print(minx_utm, miny_utm, maxx_utm, maxy_utm)


322238.41698047327 3049296.092977521 359553.76727829414 3081128.393714187


In [48]:
from shapely.geometry import shape, box
import requests
import os
from tqdm import tqdm

def bbox_to_geometry(bbox_str):
    coords = [float(x) for x in bbox_str.split(',')]
    return box(coords[0], coords[1], coords[2], coords[3])

df['geometry'] = df['proj:bbox'].apply(bbox_to_geometry)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:32645")

aoi_minx, aoi_miny, aoi_maxx, aoi_maxy = 322238.41698, 3049296.0929, 359553.7672,3081128.39371
aoi = box(aoi_minx, aoi_miny, aoi_maxx, aoi_maxy)
filtered_df = gdf[gdf.geometry.intersects(aoi)]

# sort by cloud coverage and off nadir
if 'tile:clouds_percent' in df.columns:
    filtered_df = df.sort_values(['tile:clouds_percent', 'view:off_nadir'])
filtered_df = df.drop_duplicates(subset=['ms_analytic'])

# check for spatial coverage to ensure minimum overlap
selected_images = []
selected_geometries = []
print(len(filtered_df),'check')
# take top 50 that provide good coverage with minimal overlap
for idx, row in filtered_df.iterrows():
    if len(selected_images) >= 400:
        break
    print(idx)
    # if selected_geometries:
    #     combined_existing = gpd.GeoSeries(selected_geometries).unary_union
    #     overlap_ratio = row.geometry.intersection(combined_existing).area / row.geometry.area
        
    #     if overlap_ratio > 0.8:
    #         continue
    
    selected_images.append(row)
    selected_geometries.append(row.geometry)

selected_df = pd.DataFrame(selected_images)
os.makedirs('downloaded_tifs', exist_ok=True)
downloaded_urls = set()


def generate_filename_from_url(url):
    parts = url.split('/')
    filename = '_'.join(parts[-3:])
    return f"downloaded_tifs/{filename}"

for idx, row in tqdm(selected_df.iterrows(), total=len(selected_df)):
    url = str(row['ms_analytic'])
    print(url)
    
    if url in downloaded_urls:
        print(f"Already downloaded: {url}")
        continue

    filename = generate_filename_from_url(url)
    
    if os.path.exists(filename):
        print(f"Already downloaded: {filename}")
        continue
    
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            downloaded_urls.add(url)  # Mark this URL as downloaded
            print(f"Downloaded: {filename}")
        else:
            print(f"Failed to download: {url}, Status code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")

print(f"Downloaded {len(selected_df)} unique satellite images")



281 check
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
5

  0%|          | 0/281 [00:00<?, ?it/s]

https://maxar-opendata.s3.amazonaws.com/events/Nepal-Floods-Sept-2024/ard/45/031331122122/2022-03-26/10300100D0939200-ms.tif


  0%|          | 1/281 [00:10<49:21, 10.58s/it]

Downloaded: downloaded_tifs/031331122122_2022-03-26_10300100D0939200-ms.tif
https://maxar-opendata.s3.amazonaws.com/events/Nepal-Floods-Sept-2024/ard/45/031331122123/2022-03-26/10300100D0939200-ms.tif


...

Downloaded: downloaded_tifs/031331300131_2022-03-26_10300100D0939200-ms.tif
https://maxar-opendata.s3.amazonaws.com/events/Nepal-Floods-Sept-2024/ard/45/031331301000/2022-03-26/10300100D0939200-ms.tif


 14%|█▎        | 38/281 [07:12<20:54,  5.16s/it]

Downloaded: downloaded_tifs/031331301000_2022-03-26_10300100D0939200-ms.tif
https://maxar-opendata.s3.amazonaws.com/events/Nepal-Floods-Sept-2024/ard/45/031331301002/2022-03-26/10300100D0939200-ms.tif


 14%|█▎        | 38/281 [07:13<46:11, 11.41s/it]


KeyboardInterrupt: 

In [5]:
geojson = f"{repo}datasets/{dataset}.geojson"

In [6]:
m = leafmap.Map()
m.add_geojson(geojson, layer_name=dataset)
gdf = gpd.read_file(geojson)
m.zoom_to_gdf(gdf)
m

TypeError: Object of type Timestamp is not JSON serializable